[View in Colaboratory](https://colab.research.google.com/github/josd/eye/blob/master/i2i/transduction_roots/observation_prediction_roots.ipynb)

# Transduction from observation to prediction for roots of polynomial

## Introduction

What is [Transduction (machine learning)](https://en.wikipedia.org/wiki/Transduction_(machine_learning%29):

> In logic, statistical inference, and supervised learning, transduction or
transductive inference is reasoning from observed, specific (training) cases
to specific (test) cases. In contrast, induction is reasoning from observed
training cases to general rules, which are then applied to the test cases.
The distinction is most interesting in cases where the predictions of the
transductive model are not achievable by any inductive model. Note that this
is caused by transductive inference on different test sets producing mutually
inconsistent predictions.

What is the Tensor2Tensor [Transformer model](https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/models/transformer.py):

> The Transformer model consists of an encoder and a decoder. Both are stacks
of self-attention layers followed by feed-forward layers. This model yields
good results on a number of problems, especially in NLP and machine translation.
See "Attention Is All You Need" (https://arxiv.org/abs/1706.03762) for the full
description of the model and the results obtained with its early version.

![Transformer model](https://pbs.twimg.com/media/DCKhefrUMAE9stK.jpg)

> The encoder is composed of a stack of N identical layers. Each layer has
two sub-layers. The first is a multi-head self-attention mechanism, and the
second is a simple, positionwise fully connected feed-forward network.
There is a residual connection around each of the two sub-layers, followed by
layer normalization.

> The decoder is also composed of a stack of N identical layers. In addition
to the two sub-layers in each encoder layer, the decoder inserts a third
sub-layer, which performs multi-head attention over the output of the encoder
stack. The self-attention sub-layer in the decoder stack is modified to prevent
positions from attending to subsequent positions.  This masking, combined with
the fact that the output embeddings are offset by one position, ensures that the
predictions for position i can depend only on the known outputs at positions
less than i.

In [2]:
# Preparation

# install tensor2tensor
! pip install -q -U tensor2tensor

# show versions
! pip show tensorflow
! pip show tensor2tensor

# get the needed resources
! curl -O http://josd.github.io/eye/i2i/transduction_roots/observation_prediction_roots.sh
! curl -O http://josd.github.io/eye/i2i/transduction_roots/observation_prediction_roots.py
! curl -O http://josd.github.io/eye/i2i/transduction_roots/__init__.py
! curl -O http://josd.github.io/eye/i2i/transduction_roots/test_roots.observation
! chmod +x observation_prediction_roots.sh

# clear data and model
% rm -fr /tmp/t2t_data/observation_prediction_roots/
% rm -fr /tmp/t2t_train/observation_prediction_roots/transformer-transformer_small/

# start tensorboard
! curl -O https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -o ngrok-stable-linux-amd64.zip
get_ipython().system_raw('tensorboard --logdir /tmp/t2t_train/observation_prediction_roots/transformer-transformer_small --host 0.0.0.0 --port 6006 &')
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Name: tensorflow
Version: 1.9.0rc2
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: six, wheel, setuptools, termcolor, numpy, astor, absl-py, protobuf, gast, tensorboard, grpcio
Required-by: 
Name: tensor2tensor
Version: 1.6.6
Summary: Tensor2Tensor
Home-page: http://github.com/tensorflow/tensor2tensor
Author: Google Inc.
Author-email: no-reply@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: tqdm, numpy, future, google-api-python-client, scipy, gym, gevent, flask, h5py, gunicorn, sympy, bz2file, six, requests, oauth2client
Required-by: 
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1267  100  1267    0     0   1267      0  0:00:0

In [3]:
# See the observation_prediction_roots problem

! pygmentize -g observation_prediction_roots.py

import random
import cmath
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

@registry.register_problem
class ObservationPredictionRoots(text_problems.Text2TextProblem):
  """Transduction from observation to prediction for roots of polynomial ax**2 + bx + c = 0"""

  @property
  def approx_vocab_size(self):
    return 2**14  # ~16k

  @property
  def is_generate_per_split(self):
    # generate_data will shard the data into TRAIN and EVAL for us.
    return False

  @property
  def dataset_splits(self):
    """Splits of data to produce and number of output shards for each."""
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 7,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 3,
    }]

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    del data_dir
    del tmp_dir
    del dataset_split

    for n in range(100000):
      # co

In [4]:
# See the observation_prediction_roots script

! pygmentize -g observation_prediction_roots.sh

#!/bin/bash
PROBLEM=observation_prediction_roots
MODEL=transformer
HPARAMS=transformer_small

USER_DIR=$PWD
DATA_DIR=/tmp/t2t_data/$PROBLEM
TMP_DIR=/tmp/t2t_datagen/$PROBLEM
TRAIN_DIR=/tmp/t2t_train/$PROBLEM/$MODEL-$HPARAMS

mkdir -p $DATA_DIR $TMP_DIR $TRAIN_DIR

# generate data
t2t-datagen \
  --data_dir=$DATA_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --tmp_dir=$TMP_DIR

# train with Adam for 3600 steps
t2t-trainer \
  --data_dir=$DATA_DIR \
  --eval_steps=10 \
  --eval_throttle_seconds=30 \
  --hparams_set=$HPARAMS \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --train_steps=3600

# train with SGD for 3600 steps
t2t-trainer \
  --data_dir=$DATA_DIR \
  --eval_steps=10 \
  --eval_throttle_seconds=30 \
  --hparams="optimizer=SGD" \
  --hparams_set=$HPARAMS \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --train_steps=7200

# decode
t2t-decoder \
  --data_dir=$D

In [5]:
# Run the observation_prediction_roots script

! ./observation_prediction_roots.sh

INFO:tensorflow:Importing user module content from path /
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3a20997fd0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow

INFO:tensorflow:Transforming body output with symbol_modality_170_256.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-14-22:35:03
INFO:tensorflow:Graph was finalized.
2018-07-14 22:35:03.985553: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1471] Adding visible gpu devices: 0
2018-07-14 22:35:03.985652: I tensorflow/core/common_runtime/gpu/gpu_device.cc:952] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-07-14 22:35:03.985693: I tensorflow/core/common_runtime/gpu/gpu_device.cc:958]      0 
2018-07-14 22:35:03.985725: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] 0:   N 
2018-07-14 22:35:03.985939: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1084] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 10867 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7)
INFO:tensorflow:Restoring parameters from /tmp/t2t_train/observation_

INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_170_256.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-14-22:36:05
INFO:tensorflow:Graph was finalized.
2018-07-14 22:36:05.500056: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1471] Adding visible gpu devices: 0
2018-07-14 22:36:05.500155: I tensorflow/core/common_runtime/gpu/gpu_device.cc:952] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-07-14 22:36:05.500196: I tensorflow/core/common_runtime/gpu/gpu_device.cc:958]      0 
2018-07-14 22:36:05.500228: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] 0:   N 
2018-07-14 22:36:05.500435: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1084] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 10867 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7)
INFO:tensorflow:Restoring paramet

INFO:tensorflow:Graph was finalized.
2018-07-14 22:37:05.519648: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1471] Adding visible gpu devices: 0
2018-07-14 22:37:05.519746: I tensorflow/core/common_runtime/gpu/gpu_device.cc:952] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-07-14 22:37:05.519787: I tensorflow/core/common_runtime/gpu/gpu_device.cc:958]      0 
2018-07-14 22:37:05.519819: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] 0:   N 
2018-07-14 22:37:05.520012: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1084] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 10867 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7)
INFO:tensorflow:Restoring parameters from /tmp/t2t_train/observation_prediction_roots/transformer-transformer_small/model.ckpt-286
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation 

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/10]
INFO:tensorflow:Evaluation [2/10]
INFO:tensorflow:Evaluation [3/10]
INFO:tensorflow:Evaluation [4/10]
INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-07-14-22:38:10
INFO:tensorflow:Saving dict for global step 375: global_step = 375, loss = 0.69883233, metrics-observation_prediction_roots/targets/accuracy = 0.8333869, metrics-observation_prediction_roots/targets/accuracy_per_sequence = 0.0, metrics-observation_prediction_roots/targets/accuracy_top5 = 0.90839785, metrics-observation_prediction_roots/targets/approx_bleu_score = 0.58528864, metrics-observation_prediction_roots/targets/neg_log_perplexity = -0.79268247, metrics-observation_prediction_roots/targets/rouge_2_fscore 

...

INFO:tensorflow:Evaluation [4/10]
INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-07-14-23:40:01
INFO:tensorflow:Saving dict for global step 6942: global_step = 6942, loss = 0.038174298, metrics-observation_prediction_roots/targets/accuracy = 0.99573535, metrics-observation_prediction_roots/targets/accuracy_per_sequence = 0.9014286, metrics-observation_prediction_roots/targets/accuracy_top5 = 0.9994158, metrics-observation_prediction_roots/targets/approx_bleu_score = 0.9568782, metrics-observation_prediction_roots/targets/neg_log_perplexity = -0.09293505, metrics-observation_prediction_roots/targets/rouge_2_fscore = 0.9408016, metrics-observation_prediction_roots/targets/rouge_L_fscore = 0.9659791
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 6942: /tmp/t2t_train/observ

INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-07-14-23:40:59
INFO:tensorflow:Saving dict for global step 7067: global_step = 7067, loss = 0.038126316, metrics-observation_prediction_roots/targets/accuracy = 0.99573535, metrics-observation_prediction_roots/targets/accuracy_per_sequence = 0.9014286, metrics-observation_prediction_roots/targets/accuracy_top5 = 0.9994158, metrics-observation_prediction_roots/targets/approx_bleu_score = 0.9568782, metrics-observation_prediction_roots/targets/neg_log_perplexity = -0.0928525, metrics-observation_prediction_roots/targets/rouge_2_fscore = 0.9408016, metrics-observation_prediction_roots/targets/rouge_L_fscore = 0.9659791
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 7067: /tmp/t2t_train/observation_prediction_roots/transformer-

INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-07-14-23:41:58
INFO:tensorflow:Saving dict for global step 7192: global_step = 7192, loss = 0.038087446, metrics-observation_prediction_roots/targets/accuracy = 0.99573535, metrics-observation_prediction_roots/targets/accuracy_per_sequence = 0.9014286, metrics-observation_prediction_roots/targets/accuracy_top5 = 0.9994158, metrics-observation_prediction_roots/targets/approx_bleu_score = 0.9568782, metrics-observation_prediction_roots/targets/neg_log_perplexity = -0.09277236, metrics-observation_prediction_roots/targets/rouge_2_fscore = 0.9408016, metrics-observation_prediction_roots/targets/rouge_L_fscore = 0.9659791
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 7192: /tmp/t2t_train/observation_prediction_roots/transformer

INFO:tensorflow:Evaluation [5/10]
INFO:tensorflow:Evaluation [6/10]
INFO:tensorflow:Evaluation [7/10]
INFO:tensorflow:Evaluation [8/10]
INFO:tensorflow:Evaluation [9/10]
INFO:tensorflow:Evaluation [10/10]
INFO:tensorflow:Finished evaluation at 2018-07-14-23:42:44
INFO:tensorflow:Saving dict for global step 7200: global_step = 7200, loss = 0.03808222, metrics-observation_prediction_roots/targets/accuracy = 0.99573535, metrics-observation_prediction_roots/targets/accuracy_per_sequence = 0.9014286, metrics-observation_prediction_roots/targets/accuracy_top5 = 0.9994158, metrics-observation_prediction_roots/targets/approx_bleu_score = 0.9568782, metrics-observation_prediction_roots/targets/neg_log_perplexity = -0.092773296, metrics-observation_prediction_roots/targets/rouge_2_fscore = 0.9408016, metrics-observation_prediction_roots/targets/rouge_L_fscore = 0.9659791
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 7200: /tmp/t2t_train/observation_prediction_roots/transformer

2018-07-14 23:43:21.695657: I tensorflow/core/common_runtime/gpu/gpu_device.cc:952] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-07-14 23:43:21.695732: I tensorflow/core/common_runtime/gpu/gpu_device.cc:958]      0 
2018-07-14 23:43:21.695769: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] 0:   N 
2018-07-14 23:43:21.696124: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1084] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 10867 MB memory) -> physical GPU (device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7)
INFO:tensorflow:Restoring parameters from /tmp/t2t_train/observation_prediction_roots/transformer-transformer_small/model.ckpt-7200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference results INPUT: A_POLYNOMIAL with coefficients 2 -1 9
INFO:tensorflow:Inference results OUTPUT: A_ROOT real 0.25 imag -2.11 A_ROOT real 0.25 imag 2.11
IN

In [6]:
# See the transductions

! pygmentize -g test_roots.observation
print("->-")
! pygmentize -g test_roots.prediction

A_POLYNOMIAL with coefficients 2 -1 9
->-
A_ROOT real 0.25 imag -2.11 A_ROOT real 0.25 imag 2.11


In [0]:
import os

import tensorflow as tf

from tensor2tensor import problems
from tensor2tensor.bin import t2t_decoder  # To register the hparams set
from tensor2tensor.utils import registry
from tensor2tensor.utils import trainer_lib
from tensor2tensor.visualization import attention
from tensor2tensor.visualization import visualization

In [0]:
def call_html():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))

In [0]:
# MODEL
CHECKPOINT = os.path.expanduser('/tmp/t2t_train/observation_prediction_roots/transformer-transformer_small')

In [0]:
# HPARAMS
problem_name = 'observation_prediction_roots'
data_dir = os.path.expanduser('/tmp/t2t_data/observation_prediction_roots')
model_name = "transformer"
hparams_set = "transformer_small"

In [11]:
import observation_prediction_roots

visualizer = visualization.AttentionVisualizer(hparams_set, model_name, data_dir, problem_name, beam_size=1)

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_170_256.bottom
INFO:tensorflow:Transforming 'targets' with symbol_modality_170_256.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_170_256.top
INFO:tensorflow:Greedy Decoding


In [12]:
tf.Variable(0, dtype=tf.int64, trainable=False, name='global_step')

sess = tf.train.MonitoredTrainingSession(
    checkpoint_dir=CHECKPOINT,
    save_summaries_secs=0,
)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/t2t_train/observation_prediction_roots/transformer-transformer_small/model.ckpt-7200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 7200 into /tmp/t2t_train/observation_prediction_roots/transformer-transformer_small/model.ckpt.


In [15]:
input_sentence = "A_POLYNOMIAL with coefficients 2 -1 9"
output_string, inp_text, out_text, att_mats = visualizer.get_vis_data_from_string(sess, input_sentence)
print(output_string)

A_ROOT real 0.25 imag -2.11 A_ROOT real 0.25 imag 2.11<EOS>


## Interpreting the Visualizations
- The layers drop down allow you to view the different Transformer layers, 0-indexed of course.
  - Tip: The first layer, last layer and 2nd to last layer are usually the most interpretable.
- The attention dropdown allows you to select different pairs of encoder-decoder attentions:
  - All: Shows all types of attentions together. NOTE: There is no relation between heads of the same color - between the decoder self attention and decoder-encoder attention since they do not share parameters.
  - Input - Input: Shows only the encoder self-attention.
  - Input - Output: Shows the decoder’s attention on the encoder. NOTE: Every decoder layer attends to the final layer of encoder so the visualization will show the attention on the final encoder layer regardless of what layer is selected in the drop down.
  - Output - Output: Shows only the decoder self-attention. NOTE: The visualization might be slightly misleading in the first layer since the text shown is the target of the decoder, the input to the decoder at layer 0 is this text with a GO symbol prepreded.
- The colored squares represent the different attention heads.
  - You can hide or show a given head by clicking on it’s color.
  - Double clicking a color will hide all other colors, double clicking on a color when it’s the only head showing will show all the heads again.
- You can hover over a word to see the individual attention weights for just that position.
  - Hovering over the words on the left will show what that position attended to.
  - Hovering over the words on the right will show what positions attended to it.

In [16]:
call_html()
attention.show(inp_text, out_text, *att_mats)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>